# Logistic Regression Assignment
Implement the Logistic Regression learning by gradient ascent as described in class.
Before using logistic regression, be sure to normalize the variables of the training set
to have zero mean and standard deviation 1, and to do the exact same transformation to
the test set, using the mean and standard deviation of the training set

#### Import dependencies

In [1]:
import numpy as np
from matplotlib import pyplot as plt

#### Data loading functions

In [2]:
def get_gisette():
    path = "data/gisette/"   
    
    train_x = np.loadtxt(path+"gisette_train.data")
    train_y = np.loadtxt(path+"gisette_train.labels")
    
    valid_x = np.loadtxt(path+"gisette_valid.data")
    valid_y = np.loadtxt(path+"gisette_valid.labels")
    
    test_x = np.loadtxt(path+"gisette_test.data")

    return train_x, train_y, valid_x, valid_y, test_x



def get_dexter():
    path = "data/dexter/"

    train_x = np.loadtxt(path+"dexter_train.csv", delimiter=',')
    train_y = np.loadtxt(path+"dexter_train.labels")

    valid_x = np.loadtxt(path+"dexter_valid.csv", delimiter=',')
    valid_y = np.loadtxt(path+"dexter_valid.labels")

    return train_x, train_y, valid_x, valid_y

def get_madelon():
    path = "data/MADELON/"

    train_x = np.loadtxt(path + "madelon_train.data")
    train_y = np.loadtxt(path + "madelon_train.labels")
    test_x = np.loadtxt(path + "madelon_valid.data")
    test_y = np.loadtxt(path + "madelon_valid.labels")

    return train_x, train_y, test_x, test_y

#### Normalization function

In [3]:
def normalize(train, *args):
    print(train)
    mean = np.average(train, axis=0)
    standard_deviation = np.std(train, axis=0)
    columns = train, *args
    return tuple(np.divide(column-mean, standard_deviation, where=standard_deviation!=0)
                 for column in columns)



In [4]:
def weights(w, X, eta, y):
    W=np.array(w)[np.newaxis]
    n=W.T+eta*X.T
    m=(y-(1/(1+np.exp(np.dot(-X,w)))))
    weight_increment=np.dot(n,m)

    return weight_increment


In [5]:
def hard_threshold(w, lmbda, eta=0):
    w[w<=lmbda] = 0
    w[w>lmbda] /= 1+eta
    return w

def train_tisp(x_train, y_train, x_test, y_test):
    x_train, x_test = normalize(x_train, x_test)
    learning_rate = 1 / x_train.shape[0]
    threshold = .05

    w = np.zeros(x_train.shape[1])
    for _ in range(100):
        w = weights(w, x_train, learning_rate, y_train)
        w = hard_threshold(w, threshold)

In [6]:
def plot(error_test, error_train, features):
    plt.table(cellText=[[error_train], [error_test]], rowLabels=['Training error', 'Test error'], bbox=[0.3,-0.6, .4,.3], edges="closed" )
    plt.plot(features, error_train);
    plt.plot(features, error_test);
    plt.show()

In [7]:
train_x, train_y, test_x, test_y = get_madelon()

In [8]:
train_tisp(train_x, train_y, test_x, test_y)


[[485. 477. 537. ... 479. 475. 496.]
 [483. 458. 460. ... 492. 510. 517.]
 [487. 542. 499. ... 489. 499. 498.]
 ...
 [480. 517. 631. ... 500. 523. 481.]
 [484. 481. 505. ... 473. 527. 485.]
 [474. 493. 469. ... 489. 516. 516.]]


In [17]:
train_tisp(train_x, train_y, test_x, test_y)


In [18]:
train_tisp(train_x, train_y, test_x, test_y)


[[485. 477. 537. ... 479. 475. 496.]
 [483. 458. 460. ... 492. 510. 517.]
 [487. 542. 499. ... 489. 499. 498.]
 ...
 [480. 517. 631. ... 500. 523. 481.]
 [484. 481. 505. ... 473. 527. 485.]
 [474. 493. 469. ... 489. 516. 516.]]


ValueError: operands could not be broadcast together with shapes (500,) (500,2000) 